Допустим, что вам нужно подготовить аналитический отчет по этим отзывам — например, для производителя нового продукта этой категории. Для этого будем искать упоминания товаров в отзывах (будем считать их NE). Учтите, что упоминание может выглядеть не только как "Iphone 10", но и как "модель", "телефон" и т.п.

In [462]:
import json
import gzip
import nltk
import spacy
import gensim
import pandas as pd
import numpy as np
import scipy.cluster.hierarchy as hcluster
from nltk.collocations import *
from pprint import pprint

from random import sample

import gensim.downloader

nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

Посмотрим на данные. Я взяла датасет 5-core pet supplies

In [320]:
def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse('Pet_Supplies.json.gz'):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

df = getDF('reviews_Video_Games.json.gz')

In [321]:
df.head(5)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A14CK12J7C7JRK,1223000893,Consumer in NorCal,"[0, 0]",I purchased the Trilogy with hoping my two cat...,3.0,Nice Distraction for my cats for about 15 minutes,1294790400,"01 12, 2011"
1,A39QHP5WLON5HV,1223000893,Melodee Placial,"[0, 0]",There are usually one or more of my cats watch...,5.0,Entertaining for my cats,1379116800,"09 14, 2013"
2,A2CR37UY3VR7BN,1223000893,Michelle Ashbery,"[0, 0]",I bought the triliogy and have tested out all ...,4.0,Entertaining,1355875200,"12 19, 2012"
3,A2A4COGL9VW2HY,1223000893,Michelle P,"[2, 2]",My female kitty could care less about these vi...,4.0,Happy to have them,1305158400,"05 12, 2011"
4,A2UBQA85NIGLHA,1223000893,"Tim Isenhour ""Timbo""","[6, 7]","If I had gotten just volume two, I would have ...",3.0,You really only need vol 2,1330905600,"03 5, 2012"


In [322]:
len(df), len(df.asin.unique())

(157836, 8510)

In [323]:
df.reviewText[43273]

"I am raising my third Golden Retriever, and this time, decided to try this product -- had it handy just as my puppy arrived.  In the past, I had used a traditional crate for night, and a cage during the day.  This pen is just great with the 8 panels, as you can start with few panels, and add them as the puppy grows.  Changing the configuration takes just seconds.  This product is very well designed and constructed.  My puppy is now 5 months old, and at 45 pounds, this pen still works well.  While she has more freedom as she is more responsible, the pen is still handy for time outs, when you can't supervise her, and now at night I put this pen around her new dog bed -- to contain her in our bedroom at night.  It's fantastic.  I have piece of mind that she is safe, and she likes it -- feels secure.  I do recommend that as the puppy is being house trained, use fewer panels, to discourage accidents -- I learned that one the hard way.  This product will make your life and your puppy's life

Датасет содержит 157 836 отзывов на 8 510 товаров. Как видно из случайного отзыва, покупатель называет товар по-разному: this product, this pen, the pen и просто it. При этом название playpen, указанное на [амазоне](https://www.amazon.com/IRIS-Exercise-8-Panel-Playpen-Frosty/dp/B000FS4OYA), не употреблено вообще. Извлекать упоминания из таких данных может быть непросто

**1. (3 балла)** Предложите 3 способа найти упоминания товаров в отзывах. Например, использовать bootstrapping: составить шаблоны вида "холодильник XXX", найти все соответствующие n-граммы и выделить из них называние товара. Могут помочь заголовки и дополнительные данные с Amazon. Какие данные необходимы для каждого из способов? Какие есть достоинства/недостатки?

1. Самый очевидный и простой способ: скачать к основному датасету ещё и метаданные, взять оттуда название товара, а из названия извлечь существительные

Данные: сами отзывы и заголовки товаров  

Достоинства: просто, быстро и никаких дополнительных данных  

Недостатки: очень низкий recall, например, из приведённого выше отзыва про собачий манеж не извлеклось бы ничего, потому что автор ни разу не называет товар именно playpen


2. Для первого способа расширить возможные варианты названий товара. Во-первых, добавить общие слова типа product, item, purchase. Во-вторых, добавить синонимы и в идеале гиперонимы для слов, извлечённых из названия товара. Гиперонимы удобно доставать через WordNet, но размер словаря там меньше, чем в какой-нибудь большой векторной модели. К тому же, в векторных моделях среди соседей слова тоже могут попадать гиперонимы.

Данные: отзывы и названия + WordNet или векторная модель

Достоинства: гораздо выше recall за счёт расширения списка слов, которыми может быть назван товар

Недостатки: не для всех слов получится собрать синонимы/гиперонимы, особенно если использовать WordNet. Не ясно, сколько слов надо собрать, где поставить порог. Всё ещё не решается проблема с сокращениями типа playpen - pen

3. Альтернативный вариант для второго способа: собирать синонимы не из внешних источников, а из самих отзывов. Например, отсортировать все существительные из отзывов по косинусной близости к существительным из названий товаров и к общему слову типа product.

Данные: отзывы и названия + векторная модель

Достоинства: полученные синонимы будут более полезными, потому что они все гарантированно будут содержаться в наших отзывах

Недостатки: всё ещё не понятно, где ставить порог, какая близость будет считаться достаточной. Не все слова будут в модели

4. Ещё один альтернативный вариант, решающий проблему с порогом: использовать кластеризацию. Не вручную задавать порог, а автоматически кластеризовать существительные из отзывов и брать тот кластер или кластеры, куда вошли изначальные слова из названий и слово product.

Данные: отзывы и названия + векторная модель

Достоинства: все синонимы гарантированно будут в отзывах. Не надо мучиться с определением порога близости или необходимого числа синонимов

Недостатки: я придумала этот способ совершенно из головы, пытаясь представить, каким способом я сама, читая отзывы, пыталась бы понять, о каком товаре они написаны. Очень вероятно, что результат будет не такой классный, какой мне представляется в воображении. Надо пробовать (и playpen - pen тоже вряд ли извлечётся, но это решить кажется нереально на уровне слов, а не символов)

(5. GPT-3. Меня ужасно вдохновляет идея про few-shot и one-shot learning. Надеюсь, когда-нибудь почти любая текстовая задача будет хорошо решаться на основе пары ручных примеров)

**2. (2 балла)** Реализуйте один из предложенных вами способов.  

Я попробую сделать 4 способ. Для кластеризации использую метод иерархической кластеризации, т. к. он не требует задавать количество кластеров.

Туду:
* ✔ связать asin на текст названия
* ✔ распарсить тексты и достать все существительные (к сущ. из названия добавить product)
* ✔ сгруппировать отзывы по товарам и дальше идти отдельно по группам
* ✔ для каждого сущ достать вектора из модели
* ✔ кластеризовать результаты

Собрать нужные метаданные в датафрейм:

In [324]:
data = []
with gzip.open('meta_Pet_Supplies.json.gz') as f:
    for l in f:
        d = json.loads(l.strip())
        if d['asin'] in idxs:
            data.append({'title': d['title'],
                         'asin': d['asin']})
    
# total length of list, this number equals total number of products
print(len(data))

# first row of the list
print(data[0])

5703
{'title': 'Cat Sitter DVD Trilogy - Vol 1, Vol 2 and Vol 3', 'asin': '1223000893'}


In [325]:
df_meta = pd.DataFrame(data)
df_meta.drop_duplicates(inplace=True)
df_meta['title'] = df_meta.title.str.replace('&amp;', '&')
df_meta.head(5)

,title,asin
0,"Cat Sitter DVD Trilogy - Vol 1, Vol 2 and Vol 3",1223000893
1,LitterMaid LM900 Mega Self-Cleaning Litter Box,B00005MF9U
2,LitterMaid Universal Cat Privacy Tent (LMT100),B00005MF9V
3,LitterMaid LM500 Automated Litter Box,B00005MF9T
4,LitterMaid Waste Receptacles Automatic Litter ...,B00005MF9W


Достать и сохранить леммы существительных:

In [341]:
def get_nouns(df_col):
    nouns = []
    for doc in nlp.pipe(iter(df_col.str.lower())):
        n = [token.lemma_ for token in doc if str(token).isalpha() and token.pos_ == 'NOUN']
        nouns.append(list(set(n)))
    return nouns

In [327]:
df_meta['nouns'] = get_nouns(df_meta['title'])
# учитывать слово product как означающее целевой товар
df_meta['nouns'].apply(lambda x: x.append('product'))

df_meta.head(5)

,title,asin,nouns
0,"Cat Sitter DVD Trilogy - Vol 1, Vol 2 and Vol 3",1223000893,[product]
1,LitterMaid LM900 Mega Self-Cleaning Litter Box,B00005MF9U,"[self, box, litter, product]"
2,LitterMaid Universal Cat Privacy Tent (LMT100),B00005MF9V,"[privacy, tent, product]"
3,LitterMaid LM500 Automated Litter Box,B00005MF9T,[product]
4,LitterMaid Waste Receptacles Automatic Litter ...,B00005MF9W,"[pack, box, waste, litter, product]"


In [337]:
idxs = df_meta.asin.tolist()

df.drop(df[~df['asin'].isin(idxs)].index, inplace=True)

In [338]:
len(df)

94443

In [343]:
df['nouns'] = get_nouns(df['reviewText'])

In [344]:
df.head(5)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,nouns
0,A14CK12J7C7JRK,1223000893,Consumer in NorCal,"[0, 0]",I purchased the Trilogy with hoping my two cat...,3.0,Nice Distraction for my cats for about 15 minutes,1294790400,"01 12, 2011","[picture, tree, cat, wildlife, neighbor, yr, g..."
1,A39QHP5WLON5HV,1223000893,Melodee Placial,"[0, 0]",There are usually one or more of my cats watch...,5.0,Entertaining for my cats,1379116800,"09 14, 2013","[trouble, tv, cat, dvd, mouse, time, bird]"
2,A2CR37UY3VR7BN,1223000893,Michelle Ashbery,"[0, 0]",I bought the triliogy and have tested out all ...,4.0,Entertaining,1355875200,"12 19, 2012","[one, tv, dvds, cat, sound, triliogy, volume, ..."
3,A2A4COGL9VW2HY,1223000893,Michelle P,"[2, 2]",My female kitty could care less about these vi...,4.0,Happy to have them,1305158400,"05 12, 2011","[bit, ape, kitty, video, male]"
4,A2UBQA85NIGLHA,1223000893,"Tim Isenhour ""Timbo""","[6, 7]","If I had gotten just volume two, I would have ...",3.0,You really only need vol 2,1330905600,"03 5, 2012","[guinea, hand, vol, eye, bird, quality, fisher..."


В обоих датафреймах оставлены только пересекающиеся товары и сохранены существительные из них.


Для кластеризации надо собрать матрицу с векторами:

In [125]:
vectors = gensim.downloader.load('glove-wiki-gigaword-200')

[================================================--] 97.8% 246.6/252.1MB downloaded

In [345]:
def collect_clusters(i_clusters, vocab):
    clusters = {}
    for word_idx, cluster_idx in enumerate(i_clusters):
        if cluster_idx not in clusters:
            clusters[cluster_idx] = [vocab[word_idx]]
        else:
            clusters[cluster_idx].append(vocab[word_idx])
    return clusters

Цикл сразу для всего корпуса, а не для отдельных товаров:

In [364]:
# собрать вместе все сущ из отзывов
rev_nouns = []
for n_list in df.nouns:
    rev_nouns.extend(n_list)
# собрать слова из названий
title_nouns = []
for n_list in df_meta.nouns:
    title_nouns.extend(n_list)
# добавить слова из названий к словам из отзывов
rev_nouns.extend(title_nouns)
# оставить по одному разу и только то, что есть в модели
rev_nouns = [word for word in set(rev_nouns) if word in vectors]
# собрать матрицу векторов
arr = np.zeros((len(rev_nouns), 200))
for i in range(len(arr)):
    arr[i] = vectors[rev_nouns[i]]
# кластеризовать
i_clusters = hcluster.fclusterdata(arr, 1)
# собрать кластеры в читаемый словарь
clusters = collect_clusters(i_clusters, rev_nouns)

In [365]:
# посмотреть что получается
print(len(set(title_nouns)))
for noun in sample(list(set(title_nouns)), 20):
    print(noun)
    if noun in rev_nouns:
        print(clusters[i_clusters[rev_nouns.index(noun)]])

2129
running
['running', 'run']
fleece
['fleece']
seal
['seal', 'sealing']
sweater
['cashmere', 'turtleneck', 'sweater', 'cardigan']
burger
['mcdonalds', 'taco', 'hamburger', 'burger']
cider
['cider']
choke
['choke']
dna
['dna']
ecosystem
['wetland', 'ecosystem', 'habitat']
technology
['tech', 'technology']
chase
['chase']
silencer
['silencer']
gripper
['gripper']
purpose
['purpose']
goose
['goose']
hiking
['campground', 'camping', 'biking', 'hiking', 'campsite', 'jogging', 'backpacking']
cardio
['cardio']
advantage
['advantage']
trax
['trax']
biocube


In [356]:
print(len(i_clusters))

15537


In [373]:
clusters[i_clusters[rev_nouns.index('dog')]]

['pups',
 'newfoundland',
 'puppy',
 'brunswick',
 'kitten',
 'kittens',
 'dog',
 'pet',
 'retriever',
 'labrador',
 'cats',
 'cat']

**3. (1 балл)** Соберите n-граммы с полученными сущностями (NE + левый сосед / NE + правый сосед)

Туду:
* ✔ собрать один список существительных и их синонимов из названий
* ✔ лемматизировать отзывы
* ✔ достать из лемм-х отзывов соседей

In [376]:
entities = []
for noun in set(title_nouns):
    if noun in rev_nouns: # там то, что было в век. модели
        entities.append(noun)
        entities.extend(clusters[i_clusters[rev_nouns.index(noun)]])
        
len(entities)

5936

In [380]:
def lemmatize(df_col):
    lemmas = []
    for doc in nlp.pipe(iter(df_col.str.lower())):
        ls = [token.lemma_ for token in doc if str(token).isalpha()]
        lemmas.append(ls)
    return lemmas

In [381]:
df['lemmatizedText'] = lemmatize(df['reviewText'])

In [382]:
df.head(2)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,nouns,lemmatizedText
0,A14CK12J7C7JRK,1223000893,Consumer in NorCal,"[0, 0]",I purchased the Trilogy with hoping my two cat...,3.0,Nice Distraction for my cats for about 15 minutes,1294790400,"01 12, 2011","[picture, tree, cat, wildlife, neighbor, yr, g...","[i, purchase, the, trilogy, with, hope, -PRON-..."
1,A39QHP5WLON5HV,1223000893,Melodee Placial,"[0, 0]",There are usually one or more of my cats watch...,5.0,Entertaining for my cats,1379116800,"09 14, 2013","[trouble, tv, cat, dvd, mouse, time, bird]","[there, be, usually, one, or, more, of, -PRON-..."


In [486]:
pairs = []
for text in df['lemmatizedText'].tolist():
    for i, word in enumerate(text):
        if word in entities:
            if i != 0:
                pairs.append((text[i-1], word))
            if i != len(text)-1:
                pairs.append((word, text[i+1]))

In [487]:
len(pairs)

4532798

Список пар получился слишком длинным, поэтому, чтобы упростить дальнейшие вычисления, я удалю пары, которые точно не будут верными, и дубликаты

In [488]:
pairs = [pair for pair in pairs if 'the' not in pair and 'is' not in pair and '-PRON-' not in pair]
pairs = set(pairs)

In [501]:
len(pairs)

429109

In [502]:
print(list(pairs)[:5])

[('dog', 'for'), ('guard', 'hair'), ('filter', 'uv'), ('apple', 'blossom'), ('well', 'right')]


Гораздо лучше

**4. (3 балла)** Ранжируйте n-граммы с помощью 3 коллокационных метрик (t-score, PMI и т.д.). Не забудьте про частотный фильтр / сглаживание. Выберите лучший результат (какая метрика ранжирует выше коллокации, подходящие для отчёта).

In [491]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_words([word for l_list in df['lemmatizedText'].tolist() for word in l_list])
finder.apply_freq_filter(5)

**PMI**

In [492]:
all_bigrams_pmi = finder.score_ngrams(bigram_measures.pmi)

In [493]:
len(all_bigrams_pmi)

129046

Среди всех биграмм отобрать нужные:

In [505]:
def pick(all_bigrams):
    bigrams_only = list(map(lambda x: x[0], all_bigrams))
    picked = [pair for pair in bigrams_only if pair in pairs]
    return picked

In [506]:
picked_pmi = pick(all_bigrams_pmi)

In [508]:
pprint(picked_pmi[:15])

[('playbird', 'mansion'),
 ('penn', 'plax'),
 ('luxate', 'patella'),
 ('miller', 'forge'),
 ('choy', 'mustard'),
 ('hypo', 'allergenic'),
 ('radio', 'shack'),
 ('hockey', 'puck'),
 ('jeep', 'wrangler'),
 ('aspergillus', 'niger'),
 ('lithium', 'ion'),
 ('gulf', 'coast'),
 ('titanium', 'dioxide'),
 ('jackson', 'galaxy'),
 ('tum', 'tum')]


**Student t**

In [509]:
all_bigrams_t = finder.score_ngrams(bigram_measures.student_t)
picked_t = pick(all_bigrams_t)

In [510]:
pprint(picked_t[:15])

[('be', 'a'),
 ('this', 'be'),
 ('this', 'product'),
 ('easy', 'to'),
 ('a', 'little'),
 ('there', 'be'),
 ('be', 'very'),
 ('be', 'not'),
 ('i', 'can'),
 ('out', 'of'),
 ('i', 'buy'),
 ('a', 'lot'),
 ('play', 'with'),
 ('love', 'this'),
 ('i', 'be')]


**Likelihood ratio**

In [511]:
all_bigrams_like = finder.score_ngrams(bigram_measures.likelihood_ratio)
picked_like = pick(all_bigrams_like)

In [512]:
pprint(picked_like[:15])

[('this', 'product'),
 ('easy', 'to'),
 ('play', 'with'),
 ('be', 'a'),
 ('a', 'little'),
 ('litter', 'box'),
 ('a', 'lot'),
 ('there', 'be'),
 ('a', 'bit'),
 ('lot', 'of'),
 ('out', 'of'),
 ('year', 'old'),
 ('be', 'very'),
 ('i', 'buy'),
 ('i', 'can')]


Во все списки попали пары со словами, которые не являются существительными, и это ошибка пос-теггера spacy (хотя в прошлом году в домашке он показал лучший результат). Чтобы выполнить последний пункт, я возьму метрику pmi, туда попало меньше мусора.

**5. (1 балл)** Сгруппируйте полученные коллокации по NE, выведите примеры для 5 товаров.

In [547]:
groups = {}
for pair in picked_pmi:
    pair = list(pair)
    for i, word in enumerate(pair):
        pair.remove(word)
        if word in groups:
            groups[word].extend(pair)
        else:
            groups[word] = pair

In [548]:
len(groups)

4672

In [570]:
examples = ['toy', 'treat', 'supplement', 'leash', 'harness']

for word in examples:
    print(f'\n{word}\n---')
    print('\n'.join(groups[word][:5]))


toy
---
destroyer
poodle
fox
yorkshire
unsupervise

treat
---
motivated
maker
stuffer
dispenser
dispense

supplement
---
folic
manganous
biotin
choline
niacin

leash
---
attachment
handle
hook
length
training

harness
---
vest
strap
clip
instead
fit


Результаты вышли хуже, чем я ожидала. Возможно, из-за того, что при пос-теггинге просочилось много мусора, я не смогла адекватно оценить метрики на топ-15 коллокаций и выбрала не самую хорошую. Или, возм